# JWST Pipeline Validation Testing Notebook: MIRI LRS Slitless spectroscopy

# Step: flat_field() in Spec2Pipeline

**Instruments Affected**: MIRI

# Table of Contents

 - Imports
 - Introduction
 - Run Pipeline
 - Flat Field Step Check

# Imports

 - os for simple operating system functions
 - gwcs.wcstools for bounding box operations
 - astropy.io for opening fits files
 - inspect to get the docstring of our objects.
 - IPython.display for printing markdown output
 - jwst.datamodels for building model for JWST Pipeline
 - jwst.module.PipelineStep is the pipeline step being tested
 - matplotlib.pyplot to generate plot
 - matplotlib.patches to plot shapes
 - crds for retrieving a reference file
 - astropy.utils for Box access to data
 - glob for file searching operations

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import statistics as stat
import numpy as np
from jwst.pipeline import Detector1Pipeline, Spec2Pipeline, collect_pipeline_cfgs
from jwst.pipeline import Spec2Pipeline
from jwst.background import BackgroundStep
from jwst.assign_wcs import AssignWcsStep
from jwst.extract_2d import Extract2dStep
from jwst.flatfield import FlatFieldStep
from astropy.utils.data import download_file
from jwst.datamodels import ImageModel, CubeModel, RampModel
import crds
import os
import glob
from jwst import datamodels

# Introduction
For this test we are using the flat fielding step for MIRI LRS slitless (TSO) data. For more information on the pipeline step visit the links below.
  
Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/flatfield/main.html#imaging-and-non-nirspec-spectroscopic-data
  
Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/flat_field
  
## Defining Terms

 - JWST: James Webb Space Telescope 
 - MIRI: Mid-Infrared Instrument 
 - LRS: Low Resolution Spectrometer

## Retrieving data

The simulated data are stored in Box and can be retrieved from there programmatically. The simulated data were generated with MIRISim v2.3.0. The observation consists of an observation of a stellar source in the slitless view. In this scenario, one exposures is executed at the center of the field of view.

In [ ]:
mainurl ="https://data.science.stsci.edu/redirect/JWST/TSO/pipeline_testing_miri_LRS_tso/"
fname = 'pipetest_miri_lrs_tso_100G10I.fits'
fdld = download_file(mainurl+fname)
hdu = fits.open(fdld)
hdu.info()
sci_mod = RampModel(hdu)

# Run Pipeline
The simulated data were generated with MIRISim v2.3.0. The observation consists of an observation of a stellar source in the slitless subarray. In this scenario, one exposure is executed at the nominal pointing location.
In the first steps we run the Detector1 pipeline, and the Spec2 pipeline steps before flat fielding.
  
**TO DO: data to be placed into a central location such as artifactory.**

In [ ]:
if not os.path.exists('../cfg_files/'):
    os.mkdir('../cfg_files/')
    cfgs = collect_pipeline_cfgs.collect_pipeline_cfgs(dst='../cfg_files/')

In [ ]:
# KM file location
#det1 = Detector1Pipeline.call('slitless_sim/det_images/det_image_seq1_MIRIMAGE_P750Lexp1.fits', config_file='../cfg_files/calwebb_detector1.cfg', save_results=True)

# Box file location:
det1 = Detector1Pipeline.call(sci_mod, config_file='../cfg_files/calwebb_detector1.cfg', save_results=True)

The ``det1`` output corresponds to the \_rate.fits file, which has the integrations co-added. For time series observations, we proceed with the \_rateints.fits file, in which integrations are not co-added. So we find that file and load it in for further processing.

In [ ]:
rifile = glob.glob('*_rateints.fits')
print(rifile)

det1_ints = datamodels.open(rifile[0])


Now we run the assign_wcs(), extract2d() steps with this model. 

In [ ]:
awcs = AssignWcsStep.call(det1_ints, config_file='../cfg_files/assign_wcs.cfg', save_results=True)


In [ ]:
ff = FlatFieldStep.call(awcs, config_file='../cfg_files/flat_field.cfg', save_results='True')

# Flat Field Step Check
Now that we have the output from the flat fielding step, we will download the reference file from crds and perform an independent check that the step is correctly executed. We'll look at 2 different integrations (not all 10), for a sanity check. These should look qualitatively the same (but different noise so won't be identical).

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=[12,10])
im0 = ax[0].imshow(ff.data[0,:,:], origin='lower', interpolation='None')
ax[0].set_title('Flat fielded (INT 0)')

im1 = ax[1].imshow(ff.data[4,:,:], origin='lower', interpolation='None')
ax[1].set_title('Flat fielded (INT 5)')

cbar = fig.colorbar(im1, ax=ax.ravel().tolist())


In [ ]:
# plot the reference flat field as well as its DQ extension
flat_reffile = ff.meta.ref_file.flat.name
basename = crds.core.config.pop_crds_uri(flat_reffile)
filepath = crds.locate_file(basename, "jwst")
reffile =  datamodels.open(filepath)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[10,10])
ax[0].imshow(reffile.dq, origin='lower', interpolation='None')
ax[0].set_title('Flat field - DQ flags')
ax[1].imshow(reffile.data, origin='lower', interpolation='None')
ax[1].set_title('Flat field - SCI')


The plots show some cruddy pixels at the left hand edge, but these are flagged to not be used for science. 

Next verify the shapes of the reference file against that of the science data to ensure they match.

In [ ]:
print(ff.data[0].shape)
print(ff.data[4].shape)
print(reffile.shape)

assert ff.data[0].shape == reffile.shape and ff.data[4].shape == reffile.shape, "Reference file shape does not match data shape"

In [ ]:
manflat1 = awcs.data[0] / reffile.data
manflat2 = awcs.data[4] / reffile.data


In [ ]:
# flag the pixels in the flat field ref files that are "do not use"
bad = reffile.dq != 0

# perform a numeric check. Ignoring NaNs for now as these do not fully match:
for i in range(np.shape(ff.data)[0]):
    assert np.allclose(ff.data[i,:,:][~bad], (awcs.data[i,:,:]/reffile.data)[~bad], equal_nan=True), "Pipeline calibrated file doesn't match the manual check to within tolerances"

If all assert statements in this notebook PASS, then the test is successful.
  
## END
  
Authors: K. Murray (kmurray@stsci.edu), S. Kendrew (sarah.kendrew@esa.int) -- MIRI branch
Date: 31 August 2020